# 사기회사분류 

데이터 설명 : 사기회사 분류 (종속변수 : Risk 1: 사기 , 0 : 정상)

x_train:
[https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/x_train.csv](https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/x_train.csv)

y_train:
[https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/y_train.csv](https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/y_train.csv)

x_test:
[https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/x_test.csv](https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/x_test.csv)

x_label(평가용) :
[https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/y_test.csv](https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/y_test.csv)

데이터 출처 :
[https://www.kaggle.com/sid321axn/audit-data](https://www.kaggle.com/sid321axn/audit-data)
(참고, 데이터 수정)

## 하이퍼파라미터튜닝 없이

### RandomForestClassifier로 모델설정

In [64]:
import pandas as pd
import numpy as np
import warnings

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import LabelEncoder
Scaler_m = MinMaxScaler()
Scaler_s = StandardScaler()
LE = LabelEncoder()

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
from sklearn.metrics import roc_auc_score

X_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/x_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/y_train.csv')
X_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/x_test.csv')

# print(X_train.head())
# print(y_train.head())
# print(X_test.head())
df = pd.merge(X_train,y_train, on ='ID')
# print(df.info())
X = df.drop(['ID','Risk'],axis=1)
y = df.Risk
X_t = X_test.drop('ID',axis=1)
# print(X.describe())
# print(X['Detection_Risk'].value_counts())
# print(X.info())
# print(X['LOCATION_ID'].value_counts())

X['LOCATION_ID'] = LE.fit_transform(X['LOCATION_ID'])
X_t['LOCATION_ID'] = LE.fit_transform(X_t['LOCATION_ID'])

# print(X.isnull().sum())
# print(X_t.isnull().sum())

X =X.fillna(X['Money_Value'].mean())
X_t = X_t.fillna(X_t['Money_Value'].mean())
# print(X.isnull().sum())
# print(X_t.isnull().sum())
X_m = Scaler_m.fit_transform(X)
X_tm = Scaler_m.fit_transform(X_t)
X_s = Scaler_s.fit_transform(X)
X_ts = Scaler_s.fit_transform(X_t)


X_tr_m, X_val_m, y_tr_m, y_val_m = train_test_split(X_m,y, random_state = 43, test_size=0.2)
X_tr_s, X_val_s, y_tr_s, y_val_s = train_test_split(X_s,y, random_state = 43, test_size=0.2)

rfc.fit(X_tr_m, y_tr_m)

pred_y = rfc.predict_proba(X_tm)

y_real = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/y_test.csv')
print(roc_auc_score(y_real.Risk,pred_y[:,1]))

rfc.fit(X_tr_s,y_tr_s)
pred_y2=rfc.predict_proba(X_ts)
print(roc_auc_score(y_real.Risk,pred_y2[:,1]))

submission = pd.DataFrame({'ID':X_test.ID,'pred_proba':pred_y2[:,1]})
submission.to_csv('수험번호.csv',index=False)
check = pd.read_csv('수험번호.csv')
print(check.tail())

0.977217553688142
0.9995331465919701
      ID  pred_proba
151  759        0.14
152  762        1.00
153  763        1.00
154  765        0.01
155  772        0.99


### xgboost XGBClassifier로 모델설정

In [63]:
import pandas as pd
import numpy as np 
import warnings 
from sklearn.preprocessing import MinMaxScaler, StandardScaler
Scaler_m = MinMaxScaler()
Scaler_s = StandardScaler()
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
xgbc = XGBClassifier()
from sklearn.metrics import roc_auc_score

def scaler_m(df):
    num = df.select_dtypes(include ='number').columns
    for col in num:
        df[col] = Scaler_m.fit_transform(df[[col]])
    return df

def scaler_s(df):
    num = df.select_dtypes(include ='number').columns
    for col in num:
        df[col] = Scaler_s.fit_transform(df[[col]])
    return df
def le(df):
    obj = df.select_dtypes(include ='object').columns
    for col in obj:
        df[col] = LE.fit_transform(df[[col]])
    return df

X_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/x_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/y_train.csv')
X_test= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/x_test.csv')

df=pd.merge(X_train,y_train, on ='ID')
X = df.drop(['ID','Risk'],axis=1)
y = df.Risk
X_t = X_test.drop('ID',axis=1)

# print(X.isnull().sum())
X= X.fillna(X['Money_Value'].mean())
X_t =X_t.fillna(X_t['Money_Value'].mean())

X= le(X)
X_t= le(X_t)
# print(X_t.info())

X_m = scaler_m(X)
X_tm = scaler_m(X_t)
X_s = scaler_s(X)
X_ts = scaler_s(X_t)

X_tr_m, X_val_m, y_tr_m, y_val_m = train_test_split(X_m,y, random_state = 43, test_size=0.2)
X_tr_s, X_val_s, y_tr_s, y_val_s = train_test_split(X_s,y, random_state = 43, test_size=0.2)

xgbc.fit(X_tr_m,y_tr_m)
y_pred_m = xgbc.predict_proba(X_tm)
y_real = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/y_test.csv')
print(roc_auc_score(y_real.Risk,y_pred_m[:,1]))
xgbc.fit(X_tr_s,y_tr_m)
y_pred_s = xgbc.predict_proba(X_ts)
print(roc_auc_score(y_real.Risk,y_pred_s[:,1]))

submission = pd.DataFrame({'ID':X_test.ID,'pred_proba':y_pred_m[:,1]})
submission.to_csv('수험번호.csv',index=False)
check = pd.read_csv('수험번호.csv')
print(check.tail())

C:\Users\w0w12\.conda\envs\py3.8\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8333333333333333
0.8333333333333333
      ID  pred_proba
151  759    0.003182
152  762    0.995152
153  763    0.995152
154  765    0.003182
155  772    0.995152


## 하이퍼파라미터튜닝